In [1]:
import numpy as np
import pandas as pd
import csv
import random
import math

# Инициализация

In [2]:
drop_column = ['№п/п', 'Дата рождения', 'Учебное заведение', 'Населенный пункт по прописке', 'Основание поступления']
gender_dict = {'Мужской': 1,'Женский': 0}

# количество признаков входного слоя
feature_count = 5

# Считывание данных из файла и их подготовка

In [3]:
# подготовленный датасет
train_data = []
# датасет для тестирования
test_data = []

df = pd.read_csv('data/input.csv', sep=',')
df.drop(drop_column, axis = 1, inplace = True)

# переводим пол в числовой формат, т. к. это будет один из признаков
df['Пол'].replace(gender_dict, inplace = True) 

# приводим баллы по ЕГЭ к диапозону от 0 до 1
df.astype({'Баллы ЕГЭ': 'float'}).dtypes
df['Баллы ЕГЭ'] /= 100

fios = df['ФИО'].unique()
subjects = df['Предметы ЕГЭ'].unique()
classes = df['Специальность/направление'].unique()

for el in fios:
    student = df[df['ФИО'] == el]
    features = np.zeros((6))
    for i in range(len(subjects)):
        res = student.loc[student['Предметы ЕГЭ'] == subjects[i], 'Баллы ЕГЭ'].values
        features[i] = res[0] if len(res) else 0.01
    features[5] = student.iloc[0]['Пол']
    
    # Нормализация входного вектора
    norm_features = np.zeros((6))
    for i in range(len(features)):
        sum = 0 
        for j in range(len(features)):
            sum += features[j] ** 2
        norm_features[i] = features[i] / math.sqrt(sum)
    
    #
    temp_data = dict(fio = el, features = norm_features, 
                    result = np.where(classes == student.iloc[0]['Специальность/направление'])[0].item(0))
    
    # 90% - train, 10% - test
    if (len(train_data) / len(fios)) < 0.9:
        train_data.append(temp_data)
    else:
        test_data.append(temp_data)

# Обучение

## Кохонен

In [4]:
def getKohonenWinnerIndex(x, neuron_counts, w, s):
    result = 0
    max_net = -1
    for i in range(neuron_counts):
        net = 0
        for j in range(len(x)):
            net += x[j] * w[j][i] * s[i]
        if (net > max_net):
            result = i
            max_net = net
    return result

In [5]:
def getKohonenNormResult(x, neuron_counts, w):
    result = np.zeros((neuron_counts))
    max_index = 0
    max_net = -1
    for i in range(neuron_counts):
        net = 0
        for j in range(len(x)):
            net += x[j] * w[j][i]
        if (net > max_net):
            max_index = i
            max_net = net
        result[i] = net
    
    summ = 0
    for i in range(neuron_counts):
        if i != max_index:
            summ += result[i]
    divider = math.sqrt(summ)
    for i in range(neuron_counts):    
        if i != max_index:
            result[i] /= divider
    
    result[max_index] = 1
    return result

In [ ]:
def getNeuronNet(x, w):
    result = 0
    for i in range(len(x)):
        result += x[i] * w[i]
    return result

In [ ]:
def activation(net):
    return 1.0/(1 + np.exp(-net))

In [ ]:
def logistic_deriv(x):
    return logistic(x) * (1 - logistic(x))

In [6]:
# скорость обучения слоя Кохонена
theta = 0.5
# количество нейронов кохонена
kohonen_neuron_count = 36
# штрафы кластеров
s_w = np.ones((kohonen_neuron_count))

#инициализация начальных весов слоя Кохонена kohonen_w[x][y]
# x - индекс нейрона входного слоя
# y - индекс нейрона слоя Кохонена
kohonen_w = np.zeros((len(subjects) + 1, kohonen_neuron_count))
for i in range(len(subjects) + 1):
    for j in range(kohonen_neuron_count):
        kohonen_w[i, j] = random.uniform(0.01, 0.99)

In [7]:
# Обучение cлоя Кохонена
for student in train_data:
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(student['features'], kohonen_neuron_count, kohonen_w, s_w)
    # Увеличиваем штраф, уменьшая его коэффициент
    s_w[winner_index] *= 0.9
    # Корректировка весов нейрона победителя на слое Кохонена
    for i in range(feature_count):
        kohonen_w[i][winner_index] += theta * (student['features'][i] - kohonen_w[i][winner_index])
    # Изменение скорости обучения
    theta = theta * 0.99

[0.1777641  0.1834135  0.1258485  0.13823718 0.12206652 0.13958049
 0.22683983 0.14543434 0.18252747 0.15104363 0.11510965 0.1355727
 0.17098758 0.17202934 0.17478288 0.16891101 0.1170918  0.17880279
 0.16789703 0.20829636 0.1512194  0.14202773 0.17201397 0.10677903
 0.17210696 0.20317409 0.12540669 0.08982689 0.11579539 0.15920557
 0.09457858 0.19886937 0.15001765 0.12987654 0.1657818  1.        ]
[0.14638331 0.1287911  0.13819698 0.07317565 0.10940535 0.16670926
 0.1208843  0.07485946 0.10955541 0.14072166 0.15301713 0.17525883
 0.07254517 0.12489975 0.106733   0.11681273 0.09112531 0.09785662
 0.11726862 0.12732484 0.14516726 0.13466833 0.0807497  0.15378681
 0.08409084 0.12147236 0.1162856  0.18893123 0.11107473 0.08083991
 1.         0.13271266 0.09317097 0.16060174 0.0877416  0.12621215]
[0.18936274 0.15040854 0.1625115  0.13920854 0.16056541 0.15510867
 0.19126919 0.16374842 0.19408219 0.1742473  0.16469699 0.14096551
 0.15758614 0.14441362 0.18437583 0.14704995 0.15444001 0.145

 0.10386534 0.18610834 0.16895423 0.09863266 0.18739986 0.20138428]
[0.13484684 0.13151985 0.13757506 0.1249556  0.14086904 0.15958963
 0.12771461 0.11729259 0.10540889 0.1344807  0.17073376 0.21007561
 0.09166595 0.13671058 0.12584927 0.10578062 0.14872048 0.10119517
 0.10975169 0.12944193 0.15166777 0.16874255 0.08103934 1.
 0.12254008 0.122133   0.14365503 0.15964249 0.17648498 0.10284256
 0.19773063 0.11935172 0.11978572 0.16997452 0.09909252 0.1232966 ]
[0.17369511 0.1837243  0.13977756 0.13325676 0.13084686 0.15632128
 1.         0.14316488 0.16752085 0.15747627 0.13374042 0.1542344
 0.15450735 0.176156   0.16704305 0.16707519 0.13112491 0.17201968
 0.16046898 0.20255357 0.16330181 0.14511494 0.16171102 0.13432147
 0.17509973 0.18970461 0.13870939 0.11830742 0.14145525 0.14817894
 0.11741333 0.18549218 0.14493468 0.16094779 0.15316701 0.21644953]
[0.16383262 0.18690305 0.13021704 0.14157284 0.12310718 0.15191608
 1.         0.14093524 0.16191367 0.14973419 0.1220082  0.14245917
 

In [ ]:
# Обучение перцептрона
epoch_count = 10
hiden_layer_count = 1
hiden_neuron_count = 4
output_neuron = len(classes)

for epoch in range(epoch_count):
    for student in train_data:
        train_el = getKohonenNormResult(student['features'], kohonen_neuron_count, kohonen_w)
        
            preActivation_H[node] = np.dot(training_data[sample,:], weights_ItoH[:, node])
            postActivation_H[node] = logistic(preActivation_H[node])
    
    

In [13]:
theta

0.0

# Тестирование

In [6]:
accuracy = 0
s_w = np.ones((kohonen_neuron_count))

for student in test_data:
    print('ФИО: ' + student['fio'])
    print('Поступил на специальность: ' + classes[student['result']])
    print('Результаты нейросети:')
    # Определение нейрона победителя
    winner_index = getKohonenWinnerIndex(student['features'], kohonen_neuron_count, kohonen_w, s_w)
    best_results = [0.0, 0.0, 0.0]
    best_indexes = [0, 1, 2]
    max_res = 0.0
    max_index = 0
    for i in range(grossberg_neuron_counts):
        
        if (grossberg_w[winner_index][i] > best_results[0]):
            best_results[2] = best_results[1]
            best_results[1] = best_results[0]
            best_results[0] = grossberg_w[winner_index][i]
            best_indexes[2] = best_indexes[1]
            best_indexes[1] = best_indexes[0]
            best_indexes[0] = i
        elif (grossberg_w[winner_index][i] > best_results[1]):
            best_results[2] = best_results[1]
            best_results[1] = grossberg_w[winner_index][i]
            best_indexes[2] = best_indexes[1]
            best_indexes[1] = i
        elif (grossberg_w[winner_index][i] > best_results[2]):
            best_results[2] = grossberg_w[winner_index][i]
            best_indexes[2] = i
        
    print(classes[best_indexes[0]] + ': ' + str(grossberg_w[winner_index][best_indexes[0]] * 100) + '%')
    print(classes[best_indexes[1]] + ': ' + str(grossberg_w[winner_index][best_indexes[1]] * 100) + '%')
    print(classes[best_indexes[2]] + ': ' + str(grossberg_w[winner_index][best_indexes[2]] * 100) + '%')
    
    
    if (student['result'] in best_indexes):
        accuracy += 1
    print('--------------------------------------------------')
print ('Точность: ' + str((accuracy / len(test_data)) * 100) + '%')

ФИО: Хисамиев Харис Мурадович
Поступил на специальность: Информатика и вычислительная техника
Результаты нейросети:
Электроэнергетика и электротехника: 95.32997803840593%
Металлургия: 93.63783150145223%
Конструирование и технология электронных средств: 84.6784442185268%
--------------------------------------------------
ФИО: Хлесткова Мария Александровна
Поступил на специальность: Информатика и вычислительная техника
Результаты нейросети:
Электроэнергетика и электротехника: 98.24644623139875%
Техническая физика: 94.13942249409892%
Конструирование и технология электронных средств: 93.68956783012057%
--------------------------------------------------
ФИО: Хомич Игорь Александрович
Поступил на специальность: Конструирование и технология электронных средств
Результаты нейросети:
Конструкторско-технологическое обеспечение машиностроительных производств: 94.2817248398373%
Стандартизация и метрология: 92.3389576034454%
Программная инженерия: 88.04051320869384%
--------------------------------